<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2f22ce2f933a8681c32f16fa1cec687b3584a95fd323d7e200ccb5654b116b77
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-15 15:02:16
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: 15.59 K (0.11%)
Current PnL: -17.51 L (-11.78%)
CY Booked + Current PnL: -6.48 L (-4.36%)
-------------------
Total profit:  2.15 L
Total loss:  -19.66 L
-------------------
Total Booked + Current PnL: 20.50 L (16.85%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.78%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.40 L (58.86%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-11.34,62.0,M-LC,4.26,218810.0,17330.0,9890.0,0.26,8.60,4.52,13.51,60.0,1.75,1.57,22.28,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.97,56.0,H-LC,11.12,171941.0,15764.0,10334.0,-0.75,10.09,6.01,16.71,22.0,1.53,1.23,29.29,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,104.91,60.0,M-SC,4.71,88294.0,-12483.0,12573.0,0.60,-12.39,14.24,0.09,245.0,-0.99,0.63,16.00,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.04,115010.0,14506.0,16136.0,-0.60,14.43,14.03,30.49,79.0,0.90,0.82,51.52,MH,ATH,METALS
42,ITC,452.00,-37.50,53.0,H-LC,2.91,201835.0,1697.0,19195.0,-0.24,0.85,9.51,10.44,4.0,0.09,1.45,6.46,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-30.16,54.0,H-SC,17.97,199900.0,-54.0,97291.0,1.71,-0.03,48.67,48.63,115.0,-0.00,1.43,11.52,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.42,46.0,H-MC,2.17,136990.0,1202.0,41549.0,0.89,0.89,30.33,31.48,107.0,0.03,0.98,13.11,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.79,248695.0,964.0,74086.0,1.37,0.39,29.79,30.30,92.0,0.01,1.78,13.73,X40,ATH,ELECTRICAL


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-16.28,39.0,H-LC,5.80,301423.0,-14419.0,127080.0,0.56,-4.57,42.16,35.67,5.0,-0.11,2.16,8.66,X40N,ATH,FMCG
20,CIPLA,1795.00,-20.47,48.0,H-LC,5.86,212364.0,7864.0,33554.0,-1.56,3.85,15.80,20.25,10.0,0.23,1.52,12.91,X40N,BTT,PHARMA
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.12,229958.0,-11966.0,75242.0,-0.17,-4.95,32.72,26.15,8.0,-0.16,1.65,8.67,X40,ATH,IT
6,ASIANPAINT,3465.66,-12.67,48.0,H-LC,8.41,220387.0,-31381.0,84585.0,-1.65,-12.46,38.38,21.13,27.0,-0.37,1.58,18.55,X40,ATH,PAINTS
81,UNITDSPR,1644.00,-13.66,49.0,H-LC,8.00,226404.0,-7740.0,56375.0,0.50,-3.31,24.90,20.77,86.0,-0.14,1.62,3.38,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-3.18,60.0,H-LC,1.08,240120.0,-6031.0,85243.0,0.07,-2.45,35.50,32.18,16.0,-0.07,1.72,32.71,X200,ATH,IT
42,ITC,452.00,-37.50,53.0,H-LC,2.91,201835.0,1697.0,19195.0,-0.24,0.85,9.51,10.44,4.0,0.09,1.45,6.46,X40,NTT,FMCG
51,NESTLEIND,1377.00,-7.71,60.0,H-LC,4.13,284006.0,18580.0,38199.0,-0.32,7.00,13.45,21.40,11.0,0.49,2.04,15.33,XY25,NTT,FMCG
84,VBL,671.64,-16.28,39.0,H-LC,5.80,301423.0,-14419.0,127080.0,0.56,-4.57,42.16,35.67,5.0,-0.11,2.16,8.66,X40N,ATH,FMCG
57,RELIANCE,1533.00,-12.91,54.0,H-LC,5.86,218197.0,7831.0,20947.0,0.27,3.72,9.60,13.68,37.0,0.37,1.56,20.86,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,7969.85,2.74,63.0,H-LC,28.53,165120.0,-20975.0,233381.0,3.14,-11.27,141.34,114.13,15.0,-0.09,1.18,21.60,AR,ATH,ELECTRICAL
76,TITAGARH,1548.00,1.18,70.0,H-SC,4.96,165961.0,-26054.0,104937.0,2.25,-13.57,63.23,41.08,158.0,-0.25,1.19,40.63,XY24,NTT,ENGINEERING
82,VAIBHAVGBL,521.00,54.82,62.0,H-SC,9.12,134339.0,-48436.0,159501.0,2.21,-26.50,118.73,60.77,125.0,-0.30,0.96,20.32,XR,NTT,JEWELLERY
65,SHALBY,327.00,1077.03,69.0,M-SC,2.27,160111.0,-21032.0,66494.0,2.00,-11.61,41.53,25.10,235.0,-0.32,1.15,28.31,XY24,NTT,HEALTHCARE
54,RAJESHEXPO,518.00,1859.60,64.0,L-SC,4.96,53464.0,-83713.0,83805.0,1.94,-61.03,156.75,0.07,267.0,-1.00,0.38,32.97,OX40N,NTT,JEWELLERY


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.00,236.56,66.0,M-SC,4.75,150206.0,-32872.0,111843.0,-1.79,-17.96,74.46,43.14,230.0,-0.29,1.08,41.29,XY24,NTT,MISC
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85344.0,-28205.0,68258.0,-1.77,-24.84,79.98,35.27,268.0,-0.41,0.61,108.88,XR,NTT,HOTELS
6,ASIANPAINT,3465.66,-12.67,48.0,H-LC,8.41,220387.0,-31381.0,84585.0,-1.65,-12.46,38.38,21.13,27.0,-0.37,1.58,18.55,X40,ATH,PAINTS
8,ATULAUTO,844.00,3808.11,62.0,M-SC,6.76,121859.0,-25509.0,77332.0,-1.59,-17.31,63.46,35.16,236.0,-0.33,0.87,24.47,XY24,NTT,AUTO
20,CIPLA,1795.00,-20.47,48.0,H-LC,5.86,212364.0,7864.0,33554.0,-1.56,3.85,15.80,20.25,10.0,0.23,1.52,12.91,X40N,BTT,PHARMA


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,99.53,61.0,H-SC,12.47,126117.0,-16362.0,35880.0,0.19,-11.48,28.45,13.70,163.0,-0.46,0.90,48.85,OX40N,NTT,BANKS
78,TTKPRESTIG,770.0,104.91,60.0,M-SC,4.71,88294.0,-12483.0,12573.0,0.60,-12.39,14.24,0.09,245.0,-0.99,0.63,16.00,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,58.0,H-SC,1.82,222300.0,-47367.0,83696.0,-0.22,-17.56,37.65,13.47,138.0,-0.57,1.59,12.44,XY24,NTT,PAINTS
19,CERA,9475.0,-20.53,43.0,H-SC,2.17,144900.0,-31003.0,73030.0,-0.35,-17.63,50.40,23.89,149.0,-0.42,1.04,25.14,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-20.01,52.0,H-MC,7.85,108120.0,-23715.0,65942.0,-0.54,-17.99,60.99,32.03,98.0,-0.36,0.78,21.10,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-26.12,55.0,H-LC,13.40,294540.0,-51416.0,126269.0,0.29,-14.86,42.87,21.64,1.0,-0.41,2.11,4.34,X40,ATH,IT
40,INFY,2275.00,-17.44,53.0,H-LC,9.12,321715.0,8827.0,162852.0,-1.00,2.82,50.62,54.87,3.0,0.05,2.31,9.66,X40,BTT,IT
42,ITC,452.00,-37.50,53.0,H-LC,2.91,201835.0,1697.0,19195.0,-0.24,0.85,9.51,10.44,4.0,0.09,1.45,6.46,X40,NTT,FMCG
84,VBL,671.64,-16.28,39.0,H-LC,5.80,301423.0,-14419.0,127080.0,0.56,-4.57,42.16,35.67,5.0,-0.11,2.16,8.66,X40N,ATH,FMCG
1,ABB,7934.00,-38.60,63.0,H-LC,11.52,256728.0,-4891.0,124102.0,1.90,-1.87,48.34,45.57,7.0,-0.04,1.84,9.24,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,65.0,M-MC,6.94,228199.0,3237.0,164417.0,0.20,1.44,72.05,74.53,192.0,0.02,1.64,34.16,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.82,189050.0,24380.0,43595.0,0.30,14.81,23.06,41.28,89.0,0.56,1.36,23.97,X40N,ATH,AC
85,VOLTAS,1918.49,1.30,59.0,H-MC,2.82,212535.0,20793.0,75237.0,0.95,10.84,35.40,50.08,99.0,0.28,1.52,18.38,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.79,248695.0,964.0,74086.0,1.37,0.39,29.79,30.30,92.0,0.01,1.78,13.73,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.30,57.0,H-MC,6.38,210658.0,13630.0,75984.0,0.24,6.92,36.07,45.49,102.0,0.18,1.51,20.86,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.04,115010.0,14506.0,16136.0,-0.60,14.43,14.03,30.49,79.0,0.90,0.82,51.52,MH,ATH,METALS
37,INDIAMART,4850.92,-50.35,55.0,H-SC,12.23,138383.0,15047.0,118719.0,0.40,12.20,85.79,108.45,119.0,0.13,0.99,37.28,AR,ATH,MISC
86,WHIRLPOOL,2270.00,-40.54,52.0,M-SC,1.09,100440.0,8942.0,69806.0,-0.91,9.77,69.50,86.07,223.0,0.13,0.72,45.86,XR,NTT,DURABLES
87,WIPRO,420.00,-11.52,52.0,M-LC,5.70,155670.0,4725.0,104735.0,-0.34,3.13,67.28,72.51,53.0,0.05,1.12,9.34,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,63.0,H-MC,5.77,181653.0,1845.0,110572.0,0.66,1.03,60.87,62.52,88.0,0.02,1.30,35.46,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85344.0,-28205.0,68258.0,-1.77,-24.84,79.98,35.27,268.0,-0.41,0.61,108.88,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.67,46.0,H-SC,13.68,88696.0,-12267.0,106799.0,1.58,-12.15,120.41,93.63,148.0,-0.11,0.64,32.11,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.99,57.0,H-SC,12.84,95655.0,-2325.0,23923.0,-1.21,-2.37,25.01,22.05,152.0,-0.10,0.69,39.04,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.04,115010.0,14506.0,16136.0,-0.60,14.43,14.03,30.49,79.0,0.90,0.82,51.52,MH,ATH,METALS
8,ATULAUTO,844.00,3808.11,62.0,M-SC,6.76,121859.0,-25509.0,77332.0,-1.59,-17.31,63.46,35.16,236.0,-0.33,0.87,24.47,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3808.11,62.0,M-SC,6.76,121859.0,-25509.0,77332.0,-1.59,-17.31,63.46,35.16,236.0,-0.33,0.87,24.47,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,45.55,85344.0,-28205.0,68258.0,-1.77,-24.84,79.98,35.27,268.0,-0.41,0.61,108.88,XR,NTT,HOTELS
76,TITAGARH,1548.00,1.18,70.0,H-SC,4.96,165961.0,-26054.0,104937.0,2.25,-13.57,63.23,41.08,158.0,-0.25,1.19,40.63,XY24,NTT,ENGINEERING
50,NATIONALUM,247.44,-42.00,71.0,H-MC,5.04,115010.0,14506.0,16136.0,-0.60,14.43,14.03,30.49,79.0,0.90,0.82,51.52,MH,ATH,METALS
61,SAMMAANCAP,326.00,-170.45,61.0,M-SC,2.81,81510.0,-20700.0,114090.0,-0.26,-20.25,139.97,91.37,208.0,-0.18,0.58,32.32,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.54
1,25,42.84
2,50,73.29


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.21
LC    34.06
MC    22.73
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.57
X40      15.04
X40N     12.75
XR       11.31
XY25      9.64
AR        8.52
OX40N     8.05
X200      1.72
MH        1.69
X5K       1.44
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.38
H-LC    25.92
H-MC    19.82
M-SC    13.32
M-LC     7.09
M-MC     2.61
L-SC     1.51
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.17,-14.56,77.02
FMCG,11.67,-4.00,40.39
FINANCE,9.50,-16.16,61.66
BANKS,7.66,-12.19,63.85
MISC,7.39,-11.65,73.74
PAINTS,5.76,-14.92,32.05
ELECTRICAL,5.59,-5.43,63.53
AC,3.58,2.94,29.93
INSURANCE,3.36,-7.05,43.77


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2861006.0,22
XR,1331370.0,14
AR,1307124.0,9
X40,751195.0,10
X40N,633413.0,10
OX40N,523087.0,10
XY25,405895.0,7
SR,245462.0,2
X5K,117381.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3453327.0,29
M-SC,1387119.0,17
H-LC,1258601.0,15
H-MC,1188014.0,15
M-LC,391675.0,5
M-MC,313885.0,2
L-SC,249291.0,3
L-MC,60369.0,1
L-LC,38160.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1038324.0      6
M-SC       XY24       799862.0      7
H-SC       AR         787514.0      5
           XR         773470.0      7
H-MC       XY24       488023.0      4
H-LC       X40        468143.0      5
           AR         357483.0      2
M-MC       XY24       313885.0      2
H-SC       X40N       296920.0      4
           OX40N      248508.0      4
           SR         245462.0      2
H-LC       X40N       227343.0      4
H-MC       X40        213582.0      4
           XY25       184609.0      2
L-SC       XR         165486.0      2
M-SC       AR         162127.0      2
M-LC       XY24       159669.0      2
M-SC       OX40N      124832.0      4
M-LC       X5K        117381.0      1
M-SC       XR         116738.0      2
           XY25       114090.0      1
H-MC       XR         110572.0      1
           X40N       109150.0      2
M-LC       XR         104735.0      1
H-LC       X200        85243.0      1
L-SC       OX40N       83805.0      1
M-SC       X40         69470.0      1
H-MC       OX40N       65942.0      1
H-SC       MH          63129.0      1
H-LC       XY24        61243.0      1
L-MC       XR          60369.0      1
H-LC       XY25        59146.0      2
L-LC       XY25        38160.0      1
H-MC       MH          16136.0      1
M-LC       XY25         9890.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
